In [1]:
import os
from os.path import join
import pandas as pd
import numpy as np
import requests
import json 
import re

# Pull EC from SCRAPED DATA (Excel sheets)

## Current location

In [2]:
def data_locate(rel_path_to_input,a):
    cwd = os.getcwd()
    stem=cwd
    for i in range(a):
        stem=os.path.abspath(os.path.join(stem, '..'))
    data_loc=os.path.join(stem , rel_path_to_input)
    os.chdir(data_loc)
    sub_files = os.listdir(data_loc)
    print ("Current Directory: ", cwd)
    print ("Data location: " , data_loc)
    print (sub_files)
    return data_loc, sub_files


In [ ]:
data_loc, sub_files=data_locate("Data\Extractions", 0)



In [ ]:
def file_list(data_loc, pattern):
    #data_loc is the absolute data location
    #pattern is the type of files to be analyzed (ex. 'jpeg', 'xlsx')
    print ("Data location: " , data_loc)
    file_size=[]
    Excel_list=[]
    Species_list=[]
    total_file_count=0
    species_ct=np.zeros((len(sub_files),1))
    
    for (dirname, dirs, files) in os.walk('.'):
        for filename in files:
            if filename.endswith(pattern) :
                thefile = os.path.join(dirname,filename)
                for i in range(len(sub_files)):
                    if str(sub_files[i]) in str(dirname):
                        Species_list.append(str(sub_files[i]))
                        species_ct[i]+=1
                Excel_list.append(filename)
                #clean up EC numbers
                total_file_count+=1
                print (os.path.getsize(thefile),'KB', filename, dirname)
    print ("Number of files total: ", total_file_count)
    return (Excel_list, Species_list, species_ct)
    

In [ ]:
Excel_list, Species_list, species_ct =file_list(data_loc, '.xlsx')

In [ ]:
def Filename_to_EC(Excel_list):
    EC_=[i.replace("Final.xlsx", "") for i in Excel_list]
    EC_=[i.replace("_", ".") for i in EC_]
    print (EC_)
    return EC_

In [ ]:
EC_list=Filename_to_EC(Excel_list)

# OUTPUT 

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
def Dataframe_excel_output(output_path,n, Species_list, EC_list, Filename):
    cwd = os.getcwd()
    print ("Current Directory: ", cwd)
    print (os.path.dirname(output_path))
    stem=cwd
    #move up 
    for i in range(n):
        stem=os.path.abspath(os.path.join(stem, '..'))
    output_loc=os.path.join(stem, output_path)
    os.chdir(output_loc)
    df= pd.DataFrame({"Species": Species_list, "EC_Numbers": EC_list})
    writer = pd.ExcelWriter(str(Filename)+'.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()
    print ("File name: ", str(Filename)+'.xlsx')
    

In [ ]:
Dataframe_excel_output('Output',0, Species_list, EC_list, "EC_Species")

# Pull EC from METANET links

In [3]:
#import dataframe sheet
print (os.getcwd())
data_loc, sub_files=data_locate("Output/from_API", 0)

C:\Users\zafri\Documents\Senior_Design
Current Directory:  C:\Users\zafri\Documents\Senior_Design
Data location:  C:\Users\zafri\Documents\Senior_Design\Output/from_API
['Ecoli_database.xlsx', 'file.json', 'Recon_database.xlsx']


In [4]:
ecoli_db = pd.read_excel('Ecoli_database.xlsx')
 
print("Column headings:", ecoli_db.columns)

Column headings: Index(['BIGG_ID', 'EC_Links', 'Metabolites', 'Subsystem/Genes',
       'old_identifiers'],
      dtype='object')


In [5]:
ecoli_db.head()

,BIGG_ID,EC_Links,Metabolites,Subsystem/Genes,old_identifiers
0,12DGR120tipp,[{'MetaNetX (MNX) Equation': [{'id': 'MNXR9467...,"[[{'bigg_id': '12dgr120', 'stoichiometry': 1.0...","[[{'genes': [], 'subsystem': 'Transport, Inner...",[['12DGR120tipp']]
1,12DGR140tipp,[{'MetaNetX (MNX) Equation': [{'id': 'MNXR9467...,"[[{'bigg_id': '12dgr140', 'stoichiometry': 1.0...","[[{'genes': [], 'subsystem': 'Transport, Inner...",[['12DGR140tipp']]
2,12DGR141tipp,[{'MetaNetX (MNX) Equation': [{'id': 'MNXR9467...,"[[{'bigg_id': '12dgr141', 'stoichiometry': 1.0...","[[{'genes': [], 'subsystem': 'Transport, Inner...",[['12DGR141tipp']]
3,12DGR160tipp,[{'MetaNetX (MNX) Equation': [{'id': 'MNXR9467...,"[[{'bigg_id': '12dgr160', 'stoichiometry': 1.0...","[[{'genes': [], 'subsystem': 'Transport, Inner...",[['12DGR160tipp']]
4,12DGR161tipp,[{'MetaNetX (MNX) Equation': [{'id': 'MNXR9467...,"[[{'bigg_id': '12dgr161', 'stoichiometry': 1.0...","[[{'genes': [], 'subsystem': 'Transport, Inner...",[['12DGR161tipp']]


In [6]:
metanet_id="'MetaNetX (MNX) Equation'"
biocyc_id="'BioCyc'"
ec_id="'EC Number'"
rhea_id="'RHEA'"
kegg_id="'KEGG Reaction'"

In [7]:
for i in range(50):
    print (ecoli_db["EC_Links"][i])
    if metanet_id in ecoli_db["EC_Links"][i]:
        link_metanet=re.findall('.*?(http://identifiers.org/metanetx.reaction/[a-zA-Z0-9]*).*', ecoli_db["EC_Links"][i])
        print (link_metanet)
    if biocyc_id in ecoli_db["EC_Links"][i]:
        #x = re.findall('.*([a-zA-Z]+\s[0-9][0-9],\s[0-9][0-9][0-9][0-9]).*', line) 

        link_biocyc=re.findall('.*?(http://identifiers.org/biocyc/[a-zA-Z0-9]+:+[a-zA-Z0-9]+-+[a-zA-Z0-9]+-+[a-zA-Z0-9]*).*', ecoli_db["EC_Links"][i])
        print (link_biocyc)
    if ec_id in ecoli_db["EC_Links"][i]:
        link_ec=re.findall('.*?(http://identifiers.org/ec-code/[a-zA-Z0-9.]*).*', ecoli_db["EC_Links"][i])
        print (link_ec)
    if rhea_id in ecoli_db["EC_Links"][i]:
        link_rhea=re.findall('.*?(http://identifiers.org/rhea/[a-zA-Z0-9.]*).*', ecoli_db["EC_Links"][i])
        print (link_rhea) 
    if kegg_id in ecoli_db["EC_Links"][i]:
        link_kegg=re.findall('.*?(http://identifiers.org/kegg.reaction/[a-zA-Z0-9.]*).*', ecoli_db["EC_Links"][i])
        print (link_kegg)     

[{'MetaNetX (MNX) Equation': [{'id': 'MNXR94675', 'link': 'http://identifiers.org/metanetx.reaction/MNXR94675'}]}]
['http://identifiers.org/metanetx.reaction/MNXR94675']
[{'MetaNetX (MNX) Equation': [{'id': 'MNXR94676', 'link': 'http://identifiers.org/metanetx.reaction/MNXR94676'}]}]
['http://identifiers.org/metanetx.reaction/MNXR94676']
[{'MetaNetX (MNX) Equation': [{'id': 'MNXR94677', 'link': 'http://identifiers.org/metanetx.reaction/MNXR94677'}]}]
['http://identifiers.org/metanetx.reaction/MNXR94677']
[{'MetaNetX (MNX) Equation': [{'id': 'MNXR94678', 'link': 'http://identifiers.org/metanetx.reaction/MNXR94678'}]}]
['http://identifiers.org/metanetx.reaction/MNXR94678']
[{'MetaNetX (MNX) Equation': [{'id': 'MNXR94679', 'link': 'http://identifiers.org/metanetx.reaction/MNXR94679'}]}]
['http://identifiers.org/metanetx.reaction/MNXR94679']
[{'MetaNetX (MNX) Equation': [{'id': 'MNXR94680', 'link': 'http://identifiers.org/metanetx.reaction/MNXR94680'}]}]
['http://identifiers.org/metanetx.r

In [13]:
bigg_id_list=ecoli_db["BIGG_ID"]
link_metanet_list=[]
link_biocyc_list=[]
link_ec_list=[]
link_rhea_list=[]
link_kegg_list=[]
list_of_links=[bigg_id_list,link_metanet_list,link_biocyc_list,link_ec_list,link_rhea_list,link_kegg_list]

for i in range(len(ecoli_db["BIGG_ID"])):
    if metanet_id in ecoli_db["EC_Links"][i]:
        link_metanet=re.findall('.*?(http://identifiers.org/metanetx.reaction/[a-zA-Z0-9]*).*', ecoli_db["EC_Links"][i])
        link_metanet_list.append(link_metanet)
    else:
        link_metanet_list.append("NA")         
    if biocyc_id in ecoli_db["EC_Links"][i]:
        link_biocyc=re.findall('.*?(http://identifiers.org/biocyc/[a-zA-Z0-9]+:+[a-zA-Z0-9]+-+[a-zA-Z0-9]+-+[a-zA-Z0-9]*).*', ecoli_db["EC_Links"][i])
        link_biocyc_list.append(link_biocyc)
    else:
        link_biocyc_list.append("NA")           
    if ec_id in ecoli_db["EC_Links"][i]:
        link_ec=re.findall('.*?(http://identifiers.org/ec-code/[a-zA-Z0-9.]*).*', ecoli_db["EC_Links"][i])
        link_ec_list.append(link_ec)
    else:
        link_ec_list.append("NA")
    if rhea_id in ecoli_db["EC_Links"][i]:
        link_rhea=re.findall('.*?(http://identifiers.org/rhea/[a-zA-Z0-9.]*).*', ecoli_db["EC_Links"][i])
        link_rhea_list.append(link_rhea) 
    else:
        link_rhea_list.append("NA")
    if kegg_id in ecoli_db["EC_Links"][i]:
        link_kegg=re.findall('.*?(http://identifiers.org/kegg.reaction/[a-zA-Z0-9.]*).*', ecoli_db["EC_Links"][i])
        link_kegg_list.append(link_kegg)    
    else:
        link_kegg_list.append("NA")
    

In [14]:
#sanity Check

for list_ in list_of_links:
    print (len(list_))

2583
2583
2583
2583
2583
2583


In [20]:
d = {'bigg_id_list':bigg_id_list,'link_metanet_list':link_metanet_list,'link_biocyc_list':link_biocyc_list,\
     'link_ec_list':link_ec_list,'link_rhea_list':link_rhea_list,'link_kegg_list':link_kegg_list}
df_links = pd.DataFrame(d)





In [21]:
df_links.head()

,bigg_id_list,link_biocyc_list,link_ec_list,link_kegg_list,link_metanet_list,link_rhea_list
0,12DGR120tipp,NA,NA,NA,[http://identifiers.org/metanetx.reaction/MNXR...,NA
1,12DGR140tipp,NA,NA,NA,[http://identifiers.org/metanetx.reaction/MNXR...,NA
2,12DGR141tipp,NA,NA,NA,[http://identifiers.org/metanetx.reaction/MNXR...,NA
3,12DGR160tipp,NA,NA,NA,[http://identifiers.org/metanetx.reaction/MNXR...,NA
4,12DGR161tipp,NA,NA,NA,[http://identifiers.org/metanetx.reaction/MNXR...,NA


In [25]:
Filename="EC_db_links"
writer = pd.ExcelWriter(str(Filename)+'.xlsx', engine='xlsxwriter')
df_links.to_excel(writer, sheet_name='Sheet1')
writer.save()
print ("File name: ", str(Filename)+'.xlsx')

File name:  EC_db_links.xlsx


In [23]:
cwd=os.getcwd()
print (cwd)

C:\Users\zafri\Documents\Senior_Design\Output\from_API
